In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")
print(os.getcwd())

C:\Users\Milosz\Desktop\python\thesis-recsys


In [2]:
import pandas as pd
import numpy as np
import functools
import operator
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import namedtuple

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import pad
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from src.models import NCF, DeepFM
from src.feature_store import FeatureStore, SparseFeat, DenseFeat
from src.utils import attr2tensor

torch.set_printoptions(precision=2, sci_mode=False)
torch.manual_seed(0)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
import pickle
with open("data/steam/data.pkl", "rb") as f:
    data = pd.read_pickle(f)

In [5]:
user_col = 'user_id'
item_col = 'app_id'
train_set = data['train_set'][[user_col, item_col]].values.T
supervision_set = data['supervision_set'][[user_col, item_col]].values.T
valid_set = data['valid_set'][[user_col, item_col]].values.T
user_attr = data['user_attr']
item_attr = data['item_attr']

train_set = np.concatenate([train_set, supervision_set], axis=1)

In [6]:
# scaler_user = StandardScaler()
# user_attr_preprocess = scaler_user.fit_transform(user_attr)

# scaler_item = StandardScaler()
# item_attr_preprocess = np.copy(item_attr)
# item_attr_preprocess[:, 435:] = scaler_item.fit_transform(item_attr[:, 435:])

In [7]:
def temp_item_attr_rebuild(item_attr):
    new_item_attr = []
    for row in item_attr:
        tags = row[15]
        OS = list(row[0:3].nonzero()[0])
        avg_rat = row[4:10].nonzero()[0][0]
        price_original = row[12]
        new_item_attr.append([tags, OS, avg_rat, price_original])
    return np.array(new_item_attr, dtype=object)

In [8]:
item_attr2 = temp_item_attr_rebuild(item_attr)

In [9]:
#TODO: przerobic na plik json oraz wrzucic nazwy atrybutów do słowników i zrobic liste słownikow
scheme = {
    "edge_index": {
        "user_id": {
            "emb_dim": 8,
            "num_emb": user_attr.shape[0],
            "max_len": 1,
            "type": 'sparse'
        },
        "item_id": {
            "emb_dim": 8,
            "num_emb": item_attr.shape[0],
            "max_len": 1,
            "type": 'sparse'
        }
    },
    "user_feat": {},
    "item_feat": {
        "tags": {
            "emb_dim": 4,
            "num_emb": 425,
            "max_len": 20,
            "type": 'sparse'
        },
        "OS": {
            "emb_dim": 4,
            "num_emb": 3,
            "max_len": 3,
            "type": 'sparse'
        },
        "AvgCatRating": {
            "emb_dim": 4,
            "num_emb": 6,
            "max_len": 1,
            "type": 'sparse'
        },
        "PriceOriginal": {
            "type": 'dense'
        }
    }
}

In [10]:
class DeepFMDataset(Dataset):
    def __init__(self, feature_store, edge_index, user_attr, item_attr, neg_sampl):
        self.edge_index = torch.tensor(edge_index) + 1
        self.user_attr = None #attr2tensor(feature_store.user_feat, user_attr)
        self.item_attr = attr2tensor(feature_store.item_feat, item_attr)
        
        self.users = self.edge_index[:, 0]
        self.items = self.edge_index[:, 1]

        self.n_users = 3066721
        self.n_items = self.item_attr.shape[0]

        self.neg_sampl = neg_sampl
    
    def __len__(self):
        return self.edge_index.shape[0]
    
    def __getitem__(self, idx):
        u_id = self.users[idx].repeat(self.neg_sampl + 1)
        i_id = torch.cat([self.items[idx].unsqueeze(0), self._approx_neg_sampl()])
        
        u_attr = torch.empty(self.neg_sampl + 1)
        i_attr = self.item_attr[i_id - 1]

        x = torch.column_stack((u_id, i_id, i_attr))
        y = torch.tensor([1] + [0] * self.neg_sampl)
        
        return x, y

    def _approx_neg_sampl(self):
        neg_i_id = torch.randint(low=0, high=self.n_items, size=(self.neg_sampl,))
        return neg_i_id

def collate_fn(batch):
    xs, ys = [], []
    for x, y in batch:
        xs.append(x)
        ys.append(y)
    xs = torch.cat(xs)
    ys = torch.cat(ys).to(torch.float)
    return xs, ys.unsqueeze(1)

In [11]:
def train(n_epochs, print_loss=500):
    model.train()
    batch_size = train_loader.batch_size
    
    for epoch in range(n_epochs):
        running_loss = 0.
        preds, ground_truths = [], []
        for i_batch, (batch, y_true) in enumerate(tqdm(train_loader)):
            batch, y_true = batch.to(device), y_true.to(device)
            
            y_pred = model(batch)
            loss = criterion(y_pred, y_true)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            preds.append(y_pred)
            ground_truths.append(y_true)
            running_loss += loss.item()
            
            if not ((i_batch+1) % print_loss):
                pred = torch.cat(preds, dim=0).detach().sigmoid().cpu().numpy()
                ground_truth = torch.cat(ground_truths, dim=0).detach().cpu().numpy()
                last_loss = running_loss / print_loss
                
                train_roc_auc = roc_auc_score(ground_truth, pred)
                test_loss, test_roc_auc = test()
                
                preds, ground_truths = [], []
                running_loss = 0.
                
                print(f"batch <{i_batch}>\ntrain_loss: {last_loss} - train_roc_auc: {train_roc_auc}\ntest_loss: {test_loss} - test_roc_auc: {test_roc_auc}\n")
        print(f"Epoch: {epoch}, Loss: {running_loss / len(train_loader):.4f}")

In [12]:
@torch.no_grad()
def test():
    model.eval()
    batch_size = val_loader.batch_size
    
    running_loss = 0.
    preds, ground_truths = [], []

    for i_batch, (batch, y_true) in enumerate(val_loader):
        batch, y_true = batch.to(device), y_true.to(device)
        
        y_pred = model(batch)
        loss = criterion(y_pred, y_true)
        
        preds.append(y_pred)
        ground_truths.append(y_true)
        running_loss += loss.item()
        
    pred = torch.cat(preds, dim=0).sigmoid().cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    
    test_loss = running_loss / len(val_loader)
    test_score = roc_auc_score(ground_truth, pred)

    return test_loss, test_score

In [13]:
feature_store = FeatureStore(scheme)
train_dataset = DeepFMDataset(feature_store, train_set.T, user_attr, item_attr2, neg_sampl=2)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=1024, collate_fn=collate_fn, drop_last=True)
val_dataset = DeepFMDataset(feature_store, valid_set.T, user_attr, item_attr2, neg_sampl=2)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1024, collate_fn=collate_fn, drop_last=True)

model = DeepFM(feature_store, hidden_dim=[128, 64], device=device)
#model = NCF(feature_store, hidden_dim=[64, 32, 8])
model = model.to(device)

In [14]:
criterion = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
optimizer = torch.optim.RMSprop(params=model.parameters(), lr=1e-4, momentum=0.9)

In [ ]:
train(n_epochs=10, print_loss=500)

In [15]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    
def load_model(path):
    model = DeepFM(feature_store, hidden_dim=[128, 64], device=device)
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    return model

In [16]:
#save_model(model, "models/deepfm_01.pth")

In [17]:
model = load_model("models/deepfm_01.pth")

In [18]:
def precision_k(reco_relevance, relevance, k=10):
    v = np.asarray(relevance.sum(axis=1).flatten(), dtype=int)[0].clip(1, k)
    bool_2d = np.vstack([np.concatenate((np.ones(i), np.zeros(k - i))) for i in v]).astype(bool)
    
    prec_k = (reco_relevance.getA().sum(axis=1, where=bool_2d) / v).mean()
    return prec_k

# def mean_average_prec(reco_relevance):
#     K = reco_relevance.shape[1]
    
#     mean_ap = 0.0
#     for k in range(1, K+1):
#         mean_ap += prec_k(reco_relevance[:, :k]) # DODAC MNOŻNIK 1/0 GDY ITEM JEST RELEWANTNY!!!
#     return mean_ap / K

def recall_k(reco_relevance, relevance, k=10):
    sum_relevant = relevance.sum(axis=1)
    return (reco_relevance.sum(axis=1) / sum_relevant).mean()

def ndcg_k(reco_relevance, relevance, k=10):
    v = np.asarray(relevance.sum(axis=1).flatten(), dtype=int)[0].clip(1, k)
    ideal_relevance = np.vstack([np.concatenate((np.ones(i), np.zeros(k - i))) for i in v])
    
    discount = 1 / np.log2(np.arange(2, k+2))
    idcg = (ideal_relevance * discount).sum(axis=1)
    dcg = (reco_relevance * discount).sum(axis=1)
    ndcg = (dcg / idcg).mean()
    
    return ndcg



@torch.no_grad()
def generate_embeddings(model, data):
    model.eval()
    data = data.to(device)
    x_dict = model.evaluate(data)
    return x_dict

@torch.no_grad()
def recommend_k(user_emb, item_emb, past_interactions=None, k=10, user_batch_size=1000):
    def remove_past_interactions(prob, user_batch):
        id_x = np.repeat(np.arange(user_batch.shape[0]), np.diff(past_interactions[user_batch].indptr))
        id_y = past_interactions[user_batch].indices
        prob[id_x, id_y] = -torch.inf
        return prob
    
    recommended_batches = []
    user_batches = torch.arange(user_emb.shape[0]).split(user_batch_size)
    for user_batch in user_batches:
        prob = (user_emb[user_batch] @ item_emb.T).sigmoid()
        prob = remove_past_interactions(prob, user_batch)
        recommended_batches.append(prob.topk(k, 1)[1])
    
    recommendations = torch.cat(recommended_batches, 0)
    return recommendations

def recommendation_relevance(recommendations, ground_truth):
    """
    Computes the relevance matrix of recommended items based on ground truth data.

    This function takes a matrix of recommended items and a ground truth sparse matrix, and calculates
    binary relevance of recommended items for each user. The relevance is determined by
    comparing the recommended items with the actual items in the ground truth.

    Args:
        recommendations (numpy.ndarray): A 2D matrix of shape (n_users, k) where k is the number of 
            recommended items per user. Each row contains indices representing the recommended 
            items for a user.
        ground_truth (scipy.csr_matrix): A sparse matrix of shape (n_users, n_items). The matrix 
            contains binary values indicating whether an item is relevant (1) or not (0) for each user.

    Returns:
        numpy.matrix: A 2D matrix of shape (n_users, k) containing the relevance scores of the
        recommended items for each user.
        
    Raises:
        ValueError: If the dimensions of 'recommendations' and 'ground_truth' do not match or
            are incompatible for matrix operations.
    """
    n_users, n_items = ground_truth.shape
    k = recommendations.shape[1]
    
    if recommendations.shape[0] != n_users:
        raise ValueError("Number of users in 'recommendations' should match 'ground_truth'.")
    
    user_idx = np.repeat(np.arange(n_users), k)
    item_idx = recommendations.flatten()
    relevance = ground_truth[user_idx, item_idx].reshape((n_users, k))  # get values under arrays of indices 
                                                                        # (user_idx and item_idx) from ground truth
    relevance_mask = np.asarray((ground_truth.sum(axis=1) != 0)).ravel()
    
    return relevance, relevance_mask

def evaluate_nn(model, mp_matrix, val_matrix, k):
    x_emb = generate_embeddings(model, val_data)
    recommendations = recommend_k(x_emb['user'], x_emb['app'], past_interactions=mp_matrix, 
                                  k=10, user_batch_size=10000).cpu().numpy()
    reco_relevance, relevance_mask = recommendation_relevance(recommendations, val_matrix)
    
    prec_k = precision_k(reco_relevance[relevance_mask], val_matrix[relevance_mask], k)
    rec_k = recall_k(reco_relevance[relevance_mask], val_matrix[relevance_mask], k)
    n_k = ndcg_k(reco_relevance[relevance_mask].getA(), val_matrix[relevance_mask], k)

    return {f"precision@{k}": prec_k, f"recall@{k}": rec_k, f"ndcg@{k}": n_k}

In [19]:
with open("data/steam/graph.pkl", "rb") as f:
    graph = pd.read_pickle(f)

In [20]:
mp_matrix = graph['mp_matrix']
val_matrix = graph['val_matrix']

In [21]:
@torch.no_grad()
def evaluate(model, x):
    model.eval()
    x = x.to(device)
    y = model(x)
    
    return y.sigmoid()

In [22]:
past_interactions = mp_matrix
def remove_past_interactions(prob, user_batch):
    id_x = np.repeat(np.arange(user_batch.shape[0]), np.diff(past_interactions[user_batch].indptr))
    id_y = past_interactions[user_batch].indices
    prob[id_x, id_y] = -torch.inf
    return prob

In [23]:
recommended_batches = []
user_batches = (torch.arange(3066721) + 1).split(1000)
item_ids_tensor = torch.arange(1231).unsqueeze(1) + 1
item_attr_tensor = attr2tensor(feature_store.item_feat, item_attr2)
item_tensor = torch.cat([item_ids_tensor, item_attr_tensor], dim=1)
item_tensor = item_tensor.repeat(1000, 1)
# for user_batch in tqdm(user_batches):
#     X = torch.cat([user_batch.repeat(1231, 1).t().reshape(-1, 1), item_tensor], dim=1)
#     prob = evaluate(model, X)
#     prob = prob.view(1000, -1)
#     prob = remove_past_interactions(prob, user_batch)
#     recommended_batches.append(prob.topk(10, 1)[1])
    
# recommendations = torch.cat(recommended_batches, 0)

In [ ]:
from reco_env import RecoEnv
from utils import import_data_for_env
import gym

In [ ]:
env = gym.make(RecoEnv.id, **import_data_for_env())

In [ ]:
vc = rec.user_id.value_counts()

In [ ]:
vc

In [ ]:
vc[vc >= 3]

In [ ]:
plt.plot(vc)